In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator, RegressorMixin

In [2]:
# data loading
boston=load_boston()
print(boston.data.shape)

(506, 13)


In [3]:
# checking features
print(boston.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [4]:
# setting X and y
mean=np.mean(boston.data, axis=0)
std=np.std(boston.data, axis=0)
boston_X=(boston.data - mean)/std
boston_y=boston.target

In [5]:
# making algorism
class admm_for_lasso(BaseEstimator, RegressorMixin):
    
    # initial setting
    def __init__(self, lambda_=1.0, rho=1.0, max_repeat=1000, fit_intercept=True):
        self.lambda_=lambda_ # regularized coefficient
        self.rho=rho # penalty coefficient
        self.max_repeat=max_repeat # the number of repeats
        self.fit_intercept=fit_intercept # presence of the intercept
        self.coef_=None # variable for coefficients
        self.intercept_=0.0 # variable for intercept
    
    # preparing for thresholding
    def _soft_thresholding_operator(self, x):
        alpha=self.lambda_ / self.rho
        y = np.zeros(x.shape)
        
        y[x>alpha]= x[x>alpha] - alpha
        y[x<alpha]= x[x<alpha] + alpha
        y[abs(x)<=alpha]= 0.0
        return y
    
    # fitting to model
    def fit(self, X, y):
        samples=X.shape[0]
        columns=X.shape[1]
        
        # initial values
        beta=np.dot(X.T, y)
        theta=beta.copy()
        mu=np.zeros(len(beta))
        
        #　calculating for Regularization
        for iteration in range(self.max_repeat):
            beta=np.dot(np.linalg.inv(np.dot(X.T, X) + self.rho*np.identity(columns)), np.dot(X.T, y) + self.rho*theta - mu)
            theta=self._soft_thresholding_operator(beta + mu/self.rho)
            mu+=self.rho*(beta - theta)
            
        # making coefficients and intercept
        self.coef_=beta
        if self.fit_intercept:
            self.intercept_=np.sum(y - np.dot(X[:,1:], np.zeros(columns)[1:]))/samples
        
        return self
        
    # calculating for prediction
    def predict(self, X):
        y=np.dot(X, self.coef_) + self.intercept_
        return y

In [6]:
# preparing for model fitting and prediction
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(boston_X, boston_y, train_size = 0.7, test_size = 0.3, random_state = 0)

In [7]:
# format setting
np.set_printoptions(formatter={"float": "{: 0.10f}".format})
pd.options.display.float_format = "{: 0.10f}".format

In [8]:
#creating model by ADMM algorism
model1=admm_for_lasso(lambda_=1.0,rho=1.0,max_repeat=1000, fit_intercept=True)
model1.fit(X_train, Y_train)

admm_for_lasso(fit_intercept=True, lambda_=1.0, max_repeat=1000, rho=1.0)

In [9]:
# intercept & coefficients of ADMM model
print("Lasso by ADMM")
print("intercept: " + str(model1.intercept_))
print("coefficients: ")
print(model1.coef_)

Lasso by ADMM
intercept: 22.7454802259887
coefficients: 
[-1.5974531040  3.1707736646  0.6670752228 -0.0488921855  0.3247552570
  3.0921171694  0.3806431231 -4.4033221785  4.0309235353 -6.1687487910
 -0.3717920293  1.5710610368 -4.4272968290]


In [10]:
#prediction by ADMM
y_pred1=model1.predict(X_test)
y_train_pred1 = model1.predict(X_train)
#evaluation scores
print("mean_squared_error train data: ", mean_squared_error(Y_train, y_train_pred1))
print("mean_squared_error test data: ", mean_squared_error(Y_test, y_pred1))
print("r^2 train data: ", r2_score(Y_train, y_train_pred1))
print("r^2 test data: ", r2_score(Y_test, y_pred1))

mean_squared_error train data:  27.80034281100373
mean_squared_error test data:  32.68180187883896
r^2 train data:  0.6720285214536159
r^2 test data:  0.6074988892965523


In [11]:
# by linear_model.Lasso in sklearn
from sklearn import linear_model
model2 = linear_model.Lasso(alpha=1.0, max_iter=1000)
model2.fit(X_train, Y_train)

print("Lasso by sickit-learn (coordinate descent)")
print("intercept: " + str(model2.intercept_))
print("coefficients: ")
print(model2.coef_)

y_pred2=model2.predict(X_test)
y_train_pred2 = model2.predict(X_train)

print("mean_squared_error train data: ", mean_squared_error(Y_train, y_train_pred2))
print("mean_squared_error test data: ", mean_squared_error(Y_test, y_pred2))
print("r^2 train data: ", r2_score(Y_train, y_train_pred2))
print("r^2 test data: ", r2_score(Y_test, y_pred2))

Lasso by sickit-learn (coordinate descent)
intercept: 22.5620746115923
coefficients: 
[-0.0000000000  0.0000000000 -0.0000000000  0.0000000000 -0.0000000000
  2.6760263963 -0.0000000000 -0.0000000000 -0.0000000000 -0.1459356743
 -1.7684446460  0.0000000000 -3.4173771634]
mean_squared_error train data:  26.041375932412734
mean_squared_error test data:  33.40658328282799
r^2 train data:  0.6927797392284292
r^2 test data:  0.5987944271883265


In [12]:
# coefficients comparison
df_Lasso=pd.DataFrame(index=boston.feature_names)
df_Lasso["by ADMM algorism"]=model1.coef_
df_Lasso["by sickit-learn (coordinate descent)"]=model2.coef_
df_Lasso

,by ADMM algorism,by sickit-learn (coordinate descent)
CRIM,-1.5974531040,-0.0000000000
ZN,3.1707736646,0.0000000000
INDUS,0.6670752228,-0.0000000000
CHAS,-0.0488921855,0.0000000000
NOX,0.3247552570,-0.0000000000
RM,3.0921171694,2.6760263963
AGE,0.3806431231,-0.0000000000
DIS,-4.4033221785,-0.0000000000
RAD,4.0309235353,-0.0000000000
TAX,-6.1687487910,-0.1459356743


In [13]:
# evaluation scores comparison
df_score=pd.DataFrame()
df_score["data"]=["train","test","train","test"]
df_score["mean_squared_error"]=[mean_squared_error(Y_train, y_train_pred1),mean_squared_error(Y_test, y_pred1), mean_squared_error(Y_train, y_train_pred2),mean_squared_error(Y_test, y_pred2)]
df_score["r^2"]=[r2_score(Y_train, y_train_pred1),r2_score(Y_test, y_pred1), r2_score(Y_train, y_train_pred2),r2_score(Y_test, y_pred2)]
df_score["prediction"]=["by ADMM algorism","by ADMM algorism", "by sklearn (coordinate descent)","by sklearn (coordinate descent)"]
df_score.pivot_table(index="data",columns="prediction")

mean_squared_error                                  \
prediction   by ADMM algorism by sklearn (coordinate descent)   
data                                                            
test            32.6818018788                   33.4065832828   
train           27.8003428110                   26.0413759324   

                        r^2                                  
prediction by ADMM algorism by sklearn (coordinate descent)  
data                                                         
test           0.6074988893                    0.5987944272  
train          0.6720285215                    0.6927797392